In [64]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize

In [65]:
class FashionMNISTtask1(datasets.FashionMNIST):
  def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
    super(FashionMNISTtask1, self).__init__(root, train=train, transform=transform, target_transform=target_transform,download=download)
    self.classes = self.classes[:6]

  def __getitem__(self, index):
    img, target = super(FashionMNISTtask1, self).__getitem__(index)
    if target < 6:
        return img, target
    else:
        return img, -1


In [66]:
class FashionMNISTtask2(datasets.FashionMNIST):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        super(FashionMNISTtask2, self).__init__(root, train=train, transform=transform, target_transform=target_transform,download=download)
        self.classes = self.classes[6:]

    def __getitem__(self, index):
        img, target = super(FashionMNISTtask2, self).__getitem__(index)
        if target >= 6:
            return img, target
        else:
            return img, -1

In [67]:
transform = torchvision.transforms.Compose([ToTensor(), Normalize((0.5), (0.5))])

train_dataset_1 = FashionMNISTtask1(root='./data1', train=True, transform=transform, download=True)
test_dataset_1 = FashionMNISTtask1(root='./data1', train=False, transform=transform, download=True)

In [68]:
train_dataset_2 = FashionMNISTtask2(root='./data2', train=True, transform=transform, download=True)
test_dataset_2 = FashionMNISTtask2(root='./data2', train=False, transform=transform, download=True)
test_dataset_3 = datasets.FashionMNIST(
    root='data3',
    train=False,
    download=True,
    transform=transform
)

In [69]:
print(dir(datasets.FashionMNIST))

['__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_check_exists', '_check_legacy_exist', '_format_transform_repr', '_is_protocol', '_load_data', '_load_legacy_data', '_repr_indent', 'class_to_idx', 'classes', 'download', 'extra_repr', 'mirrors', 'processed_folder', 'raw_folder', 'resources', 'test_data', 'test_file', 'test_labels', 'train_data', 'train_labels', 'training_file']


In [70]:
train_dataset_filtered_old = [data for data in train_dataset_1 if data[1] != -1]
test_dataset_filtered_old = [data for data in test_dataset_1 if data[1] != -1]

train_dataset_filtered_new = [data for data in train_dataset_2 if data[1] != -1]
test_dataset_filtered_new = [data for data in test_dataset_2 if data[1] != -1]

In [71]:
train_dataloader_old = DataLoader(train_dataset_filtered_old, batch_size=64, shuffle=True)
test_dataloader_old = DataLoader(test_dataset_filtered_old, batch_size=256, shuffle=False)

for X, y in train_dataloader_old:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"SHape of y: {y.shape}, dtype: {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
SHape of y: torch.Size([64]), dtype: torch.int64


In [72]:
len(train_dataloader_old)

563

In [73]:
train_dataloader_new = DataLoader(train_dataset_filtered_new, batch_size=64, shuffle=True)
test_dataloader_new = DataLoader(test_dataset_filtered_new, batch_size=256, shuffle=False)

In [74]:
eval_dataloader = DataLoader(test_dataset_3, batch_size=256, shuffle=True)

In [75]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


In [76]:
def kaiming_normal_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')

In [77]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(28*28, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')
            elif isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        logits = self.classifier(x)

        return logits

In [78]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [79]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}\n")


In [80]:
pre_model = NeuralNetwork(num_classes=6, hidden_size=512).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(pre_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [81]:
print(dir(nn.Linear(2, 2)))

['T_destination', '__annotations__', '__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_s

In [82]:
epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader_old, pre_model, loss_fn, optimizer)
  test(test_dataloader_old, pre_model, loss_fn)
print("Done!")

torch.save(pre_model.state_dict(), "model_old.pth")

Epoch 1
---------------------------
Loss: 1.809697,    64/36000
Loss: 0.430445,  6464/36000
Loss: 0.432301, 12864/36000
Loss: 0.354811, 19264/36000
Loss: 0.199749, 25664/36000
Loss: 0.255814, 32064/36000
Test Error: 
 Accuracy: 89.1, Avg Loss: 0.288040

Epoch 2
---------------------------
Loss: 0.270974,    64/36000
Loss: 0.309610,  6464/36000
Loss: 0.311149, 12864/36000
Loss: 0.230740, 19264/36000
Loss: 0.255295, 25664/36000
Loss: 0.417961, 32064/36000
Test Error: 
 Accuracy: 90.4, Avg Loss: 0.271922

Epoch 3
---------------------------
Loss: 0.282508,    64/36000
Loss: 0.226512,  6464/36000
Loss: 0.216481, 12864/36000
Loss: 0.094972, 19264/36000
Loss: 0.256139, 25664/36000
Loss: 0.311094, 32064/36000
Test Error: 
 Accuracy: 90.4, Avg Loss: 0.262038

Epoch 4
---------------------------
Loss: 0.212492,    64/36000
Loss: 0.123525,  6464/36000
Loss: 0.177934, 12864/36000
Loss: 0.281333, 19264/36000
Loss: 0.154010, 25664/36000
Loss: 0.178204, 32064/36000
Test Error: 
 Accuracy: 90.6, Avg 

# LWF

In [95]:
net_new = NeuralNetwork(num_classes=6, hidden_size=512).to(device)
net_old = NeuralNetwork(num_classes=6, hidden_size=512).to(device)

net_new.load_state_dict(torch.load("model_old.pth"))
net_old.load_state_dict(torch.load("model_old.pth"))

in_features = net_old.classifier.in_features
out_features = net_old.classifier.out_features

weight = net_old.classifier.weight.data
bias = net_old.classifier.bias.data

new_out_features = 6 + 4

new_fc = nn.Linear(in_features, new_out_features)
kaiming_normal_init(new_fc.weight)

new_fc.weight.data[:out_features] = weight
new_fc.bias.data[:out_features] = bias

net_new.classifier = new_fc
net_new = net_new.to(device)
print("New head numbers: ", net_new.classifier.out_features)

for param in net_old.parameters():
  param.requires_grad = False

New head numbers:  10


Changes in training and testing

In [100]:
def train(alpha, T):
  size = len(train_dataloader_new.dataset)
  # We set net_new to evaluation mode to prevent it from being updated
  # while computing the distillation loss from the old model
  net_new.train()
  for batch, (X, y) in enumerate(train_dataloader_new):
    X, y = X.to(device), y.to(device)

    outputs = net_new(X)
    soft_y = net_old(X)

    loss1 = loss_fn(outputs, y)

    outputs_S = nn.functional.softmax(outputs[:, :out_features] / T, dim=1)
    outputs_T = nn.functional.softmax(soft_y[:, :out_features] / T, dim=1)

    loss2 = outputs_T.mul(-1 * torch.log(outputs_S))
    loss2 = loss2.sum(1)
    loss2 = loss2.mean() * T * T

    loss = loss1 + alpha * loss2

    loss.backward(retain_graph=True)
    warmup_scheduler.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")


In [97]:
def test(alpha, T):
  size = len(test_dataloader_new.dataset)
  num_batches = len(test_dataloader_new)
  net_new.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in test_dataloader_new:
      X, y = X.to(device), y.to(device)

      outputs = net_new(X)
      soft_y = net_old(X)

      loss1 = loss_fn(outputs, y)

      outputs_S = nn.functional.softmax(outputs[:, :out_features] / T, dim=1)
      outputs_T = nn.functional.softmax(soft_y[:, :out_features] / T, dim=1)

      loss2 = outputs_T.mul(-1 * torch.log(outputs_S))
      loss2 = loss2.sum(1)
      loss2 = loss2.mean() * T * T

      loss = loss1 * alpha + loss2 * (1 - alpha)

      test_loss += loss.item()
      _, predicted = outputs.max(1)
      correct += predicted.eq(y).sum().item()
      # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}")


In [98]:
def val(epoch):
    net_new.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(eval_dataloader):
            X, y = X.to(device), y.to(device)
            outputs = net_new(X)
            _, predicted_old = outputs.max(1)
            total += len(y)
            correct += predicted_old.eq(y).sum().item()
        print(f"Validation Acc: {100. * correct / total}\n")

In [101]:
T = 2
alpha = 0.5
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net_new.parameters()), lr=0.01, momentum=0.9, weight_decay=5e-4)

warmup_epochs = 5
initial_lr = 0.0001
final_lr = 0.01

warmup_scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lr_lambda= lambda epoch: (epoch+1)/warmup_epochs if epoch < warmup_epochs else final_lr/initial_lr
)

for epoch in range(10):
    print(f"Epoch {epoch+1}: ----------------------")
    train(alpha, T)
    test(alpha, T)
    val(epoch)

torch.save(net_new.state_dict(), "model.pth")

Epoch 1: ----------------------
Loss: 9.623205,    64/24000


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Loss: 9.493693,  6464/24000
Loss: 9.840966, 12864/24000
Loss: 9.722804, 19264/24000
Test Error: 
 Accuracy: 0.0, Avg Loss: 5.270410
Validation Acc: 55.4

Epoch 2: ----------------------
Loss: 8.980170,    64/24000
Loss: 9.135077,  6464/24000
Loss: 9.483232, 12864/24000
Loss: 9.325359, 19264/24000
Test Error: 
 Accuracy: 0.0, Avg Loss: 5.270410
Validation Acc: 55.4

Epoch 3: ----------------------
Loss: 9.414894,    64/24000
Loss: 9.162929,  6464/24000
Loss: 9.336340, 12864/24000
Loss: 9.883241, 19264/24000
Test Error: 
 Accuracy: 0.0, Avg Loss: 5.270410
Validation Acc: 55.4

Epoch 4: ----------------------
Loss: 9.227899,    64/24000
Loss: 9.495513,  6464/24000
Loss: 9.634324, 12864/24000
Loss: 8.934695, 19264/24000
Test Error: 
 Accuracy: 0.0, Avg Loss: 5.270410
Validation Acc: 55.4

Epoch 5: ----------------------
Loss: 9.550862,    64/24000
Loss: 9.167891,  6464/24000
Loss: 9.827006, 12864/24000
Loss: 9.288913, 19264/24000
Test Error: 
 Accuracy: 0.0, Avg Loss: 5.270410
Validation A